In [1]:
!pip install torch transformers datasets evaluate accelerate memory-profiler



Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set device to CPU
device = torch.device("cpu")

# Load GPT model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"  # You can change to GPT-2 or GPT-3 variants
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)  # Move to CPU


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

C:\Users\jyoth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jyoth\.cache\huggingface\hub\models--EleutherAI--gpt-neo-1.3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
sample_text = dataset["test"][0]["article"]  # Take one sample article
print(f"Sample Text: {sample_text[:500]}...")  # Print first 500 characters


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

C:\Users\jyoth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jyoth\.cache\huggingface\hub\datasets--cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Sample Text: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, includin...


In [4]:
def measure_inference_time(text, model, tokenizer, max_new_tokens=50):
    inputs = tokenizer(text, return_tensors="pt").to(device)  # Tokenize and move to CPU

    # Measure time taken
    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    end_time = time.time()

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    time_taken = end_time - start_time
    return generated_text, time_taken

summary, inference_time = measure_inference_time(sample_text, model, tokenizer)
print(f"Inference Time (CPU): {inference_time:.2f} seconds")
print(f"Generated Summary: {summary}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Inference Time (CPU): 23.29 seconds
Generated Summary: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minis

In [5]:
from memory_profiler import memory_usage

def measure_memory_usage(text, model, tokenizer, max_new_tokens=50):
    def generate_text():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        with torch.no_grad():
            model.generate(**inputs, max_new_tokens=max_new_tokens)

    mem_used = memory_usage(generate_text, max_usage=True)
    return mem_used

memory_used = measure_memory_usage(sample_text, model, tokenizer)
print(f"Memory Used (CPU): {memory_used:.2f} MB")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Memory Used (CPU): 6553.27 MB


In [6]:
!pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=25025 sha256=289e9ba3a57b80e4364fa4990ebb00c2db719122ba3f58d186c142e130e0c7f6
  Stored in directory: c:\users\jyoth\appdata\local\packages\pythonsoftwarefoundation.python.3.12_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [7]:
import evaluate

rouge = evaluate.load("rouge")

reference_summary = dataset["test"][0]["highlights"]  # Ground truth summary
results = rouge.compute(predictions=[summary], references=[reference_summary])

print(f"ROUGE Score: {results}")


ROUGE Score: {'rouge1': 0.09480122324159022, 'rouge2': 0.05521472392638037, 'rougeL': 0.07951070336391437, 'rougeLsum': 0.08256880733944955}


In [8]:
print("=== Benchmark Results (CPU) ===")
print(f"Inference Time: {inference_time:.2f} seconds")
print(f"Memory Used: {memory_used:.2f} MB")
print(f"ROUGE Score: {results}")

=== Benchmark Results (CPU) ===
Inference Time: 23.29 seconds
Memory Used: 6553.27 MB
ROUGE Score: {'rouge1': 0.09480122324159022, 'rouge2': 0.05521472392638037, 'rougeL': 0.07951070336391437, 'rougeLsum': 0.08256880733944955}
